# Project  (Mod 1 Project - Flatiron Data Science Bootcamp, March , 2020 Washington, D.C.)

![](movies.jpg)

### STEP 1: IMPORTING LIBRARIES

### Exploring the whole dataset

To get a sense for what is in the movies dataset, first I will do some basic exploration of the entire dataset. After cleaning, another round of exploration will help clarify trends in the data specific to profit.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## STEP 2: DATA CLEANING AND PREPROCESSING

In this step we check whether data contain null or missing values. What is the size of the data. What is the datatype of each column. What are unique values of categorical variables etc

In [2]:
df_imdb = pd.read_csv('imdb.title.basics.csv.gz')
display(df_imdb.head())
display(df_imdb.info())
df_tn = pd.read_csv('tn.movie_budgets.csv.gz')
display(df_tn.head())
display(df_tn.info())
df_tmdb = pd.read_csv('tmdb.movies.csv.gz')
display(df_tmdb.head())
display(df_tmdb.info())

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
tconst             146144 non-null object
primary_title      146144 non-null object
original_title     146123 non-null object
start_year         146144 non-null int64
runtime_minutes    114405 non-null float64
genres             140736 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


None

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null object
domestic_gross       5782 non-null object
worldwide_gross      5782 non-null object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


None

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
Unnamed: 0           26517 non-null int64
genre_ids            26517 non-null object
id                   26517 non-null int64
original_language    26517 non-null object
original_title       26517 non-null object
popularity           26517 non-null float64
release_date         26517 non-null object
title                26517 non-null object
vote_average         26517 non-null float64
vote_count           26517 non-null int64
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


None

In [3]:
#get rid of money signs in df_tn "domestic_gross."
df_tn['domestic_gross'] = df_tn['domestic_gross'].str.replace('$', '')
df_tn

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",0,$0


In [4]:

#get rid of commas in df_tn "domestic_gross."
df_tn['domestic_gross'] = df_tn['domestic_gross'].str.replace(',', '')
df_tn

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000",760507625,"$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000",241063875,"$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000",42762350,"$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000",459005868,"$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000",620181382,"$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",0,$0
5778,79,"Apr 2, 1999",Following,"$6,000",48482,"$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000",1338,"$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",0,$0


In [5]:

#convert df_tn 'domestic_gross' from str to int.
df_tn['domestic_gross'] = df_tn['domestic_gross'].astype(int)
df_tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null object
domestic_gross       5782 non-null int64
worldwide_gross      5782 non-null object
dtypes: int64(2), object(4)
memory usage: 271.2+ KB


In [6]:
#Data cleaning for df_tn "production_budget"
#Get rid of money signs ($).
#Get rid of commas (,).
#Convert str to int.

In [7]:

#get rid of money signs in df_tn "production_budget."
df_tn['production_budget'] = df_tn['production_budget'].str.replace('$', '')
df_tn

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"425,000,000",760507625,"$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"410,600,000",241063875,"$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"350,000,000",42762350,"$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"330,600,000",459005868,"$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"317,000,000",620181382,"$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"7,000",0,$0
5778,79,"Apr 2, 1999",Following,"6,000",48482,"$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"5,000",1338,"$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"1,400",0,$0


In [8]:

#get rid of commas in df_tn "production_budget."
df_tn['production_budget'] = df_tn['production_budget'].str.replace(',', '')
df_tn

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,"$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,"$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,"$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,"$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,"$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,$0
5778,79,"Apr 2, 1999",Following,6000,48482,"$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,"$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,$0


In [9]:

#convert df_tn 'production_budget' from str to int.
df_tn['production_budget'] = df_tn['production_budget'].astype(int)
df_tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null int64
domestic_gross       5782 non-null int64
worldwide_gross      5782 non-null object
dtypes: int64(3), object(3)
memory usage: 271.2+ KB


In [10]:
#Data cleaning for "worldwide_gross"
#Get rid of money signs ($).
#Get rid of commas (,).
#Convert str to int.

In [11]:

#get rid of money signs in df_tn "worldwide_gross."
df_tn['worldwide_gross'] = df_tn['worldwide_gross'].str.replace('$', '')
df_tn

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,"2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,"1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,"149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,"1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,"1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0
5778,79,"Apr 2, 1999",Following,6000,48482,"240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,"1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0


In [12]:

#get rid of commas in df_tn "worldwide_gross."
df_tn['worldwide_gross'] = df_tn['worldwide_gross'].str.replace(',', '')
df_tn

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0
5778,79,"Apr 2, 1999",Following,6000,48482,240495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0


In [13]:

#get rid of commas in df_tn "worldwide_gross."
df_tn['worldwide_gross'] = df_tn['worldwide_gross'].str.replace(',', '')
df_tn

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0
5778,79,"Apr 2, 1999",Following,6000,48482,240495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0


In [14]:
#convert df_tn 'worldwide_gross' from str to int.
df_tn['worldwide_gross'] = df_tn['worldwide_gross'].astype(int)
df_tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null int64
domestic_gross       5782 non-null int64
worldwide_gross      5782 non-null int64
dtypes: int64(4), object(2)
memory usage: 271.2+ KB


In [15]:
#Data cleaning for df_tn "release_date"
#Convert str to datetime object to parse out the "month" in later analysis.

In [16]:

# convert df_tn release_date to date time object 
df_tn['release_date'] =  pd.to_datetime(df_tn['release_date'], infer_datetime_format=True)
df_tn.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,425000000,760507625,2776345279
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


In [17]:
df_tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null datetime64[ns]
movie                5782 non-null object
production_budget    5782 non-null int64
domestic_gross       5782 non-null int64
worldwide_gross      5782 non-null int64
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 271.2+ KB


In [18]:
#Data cleaning for df_tmdb "release_date"
#Convert str to datetime object to parse out the "month" in later analysis.

In [19]:
# convert df_tmdb release_date to date time object 
df_tmdb['release_date'] =  pd.to_datetime(df_tmdb['release_date'], infer_datetime_format=True)
df_tmdb.head()
df_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
Unnamed: 0           26517 non-null int64
genre_ids            26517 non-null object
id                   26517 non-null int64
original_language    26517 non-null object
original_title       26517 non-null object
popularity           26517 non-null float64
release_date         26517 non-null datetime64[ns]
title                26517 non-null object
vote_average         26517 non-null float64
vote_count           26517 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 2.0+ MB


In [20]:
#Data cleaning for value names
#Convert all columns with string values to lower case for consistency.

In [21]:

#convert all columns with string values to lower case for consistency.
df_tmdb['title'] = df_tmdb['title'].str.lower()
df_tn['movie'] = df_tn['movie'].str.lower()
df_imdb['primary_title'] = df_imdb['primary_title'].str.lower()
df_imdb['genres'] = df_imdb['genres'].str.lower()
display(df_imdb.head())
display(df_tmdb.head())
display(df_tn.head())

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,sunghursh,Sunghursh,2013,175.0,"action,crime,drama"
1,tt0066787,one day before the rainy season,Ashad Ka Ek Din,2019,114.0,"biography,drama"
2,tt0069049,the other side of the wind,The Other Side of the Wind,2018,122.0,drama
3,tt0069204,sabse bada sukh,Sabse Bada Sukh,2018,NaN,"comedy,drama"
4,tt0100275,the wandering soap opera,La Telenovela Errante,2017,80.0,"comedy,drama,fantasy"


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,harry potter and the deathly hallows: part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,how to train your dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,iron man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,toy story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,inception,8.3,22186


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,avatar,425000000,760507625,2776345279
1,2,2011-05-20,pirates of the caribbean: on stranger tides,410600000,241063875,1045663875
2,3,2019-06-07,dark phoenix,350000000,42762350,149762350
3,4,2015-05-01,avengers: age of ultron,330600000,459005868,1403013963
4,5,2017-12-15,star wars ep. viii: the last jedi,317000000,620181382,1316721747


In [22]:

#Data cleaning to eliminate unused columns of data
#We will get rid of columns of data that we are not considering in our analysis.

In [23]:

# drop unused columns in all dataframes
df_imdb_edit = df_imdb.drop(['tconst','original_title','start_year','runtime_minutes'], axis =1)
df_tn_edit = df_tn.drop(['id','domestic_gross'], axis =1)
df_tmdb_edit = df_tmdb.drop(['Unnamed: 0','genre_ids','id','original_title'], axis =1)
#drop release_date from "df_tmdb_edit" since we already have a "release_date" in "df_tn_edit"
df_tmdb_edit = df_tmdb_edit.drop('release_date', axis=1)

display(df_imdb_edit.head())
display(df_tmdb_edit.head())
display(df_tn_edit.head())

,primary_title,genres
0,sunghursh,"action,crime,drama"
1,one day before the rainy season,"biography,drama"
2,the other side of the wind,drama
3,sabse bada sukh,"comedy,drama"
4,the wandering soap opera,"comedy,drama,fantasy"


,original_language,popularity,title,vote_average,vote_count
0,en,33.533,harry potter and the deathly hallows: part 1,7.7,10788
1,en,28.734,how to train your dragon,7.7,7610
2,en,28.515,iron man 2,6.8,12368
3,en,28.005,toy story,7.9,10174
4,en,27.920,inception,8.3,22186


,release_date,movie,production_budget,worldwide_gross
0,2009-12-18,avatar,425000000,2776345279
1,2011-05-20,pirates of the caribbean: on stranger tides,410600000,1045663875
2,2019-06-07,dark phoenix,350000000,149762350
3,2015-05-01,avengers: age of ultron,330600000,1403013963
4,2017-12-15,star wars ep. viii: the last jedi,317000000,1316721747


In [24]:

#Data cleaning to eliminate duplicate rows
#We will get rid of any duplicate rows in our dataset.

In [25]:

#eliminate any duplicates in df_imdb_edit and reassign.
df_imdb_edit = df_imdb_edit.drop_duplicates(subset=None, keep='first')
display(df_imdb_edit)
#eliminate any duplicates in df_tn_edit and reassign.
df_tn_edit = df_tn_edit.drop_duplicates(subset=None, keep='first')
display(df_tn_edit)
#eliminate any duplicates in df_tn_edit and reassign.
df_tmdb_edit = df_tmdb_edit.drop_duplicates(subset=None, keep='first')
display(df_tmdb_edit)

,primary_title,genres
0,sunghursh,"action,crime,drama"
1,one day before the rainy season,"biography,drama"
2,the other side of the wind,drama
3,sabse bada sukh,"comedy,drama"
4,the wandering soap opera,"comedy,drama,fantasy"
...,...,...
146139,kuambil lagi hatiku,drama
146140,rodolpho teóphilo - o legado de um pioneiro,documentary
146141,dankyavar danka,comedy
146142,6 gunn,NaN


,release_date,movie,production_budget,worldwide_gross
0,2009-12-18,avatar,425000000,2776345279
1,2011-05-20,pirates of the caribbean: on stranger tides,410600000,1045663875
2,2019-06-07,dark phoenix,350000000,149762350
3,2015-05-01,avengers: age of ultron,330600000,1403013963
4,2017-12-15,star wars ep. viii: the last jedi,317000000,1316721747
...,...,...,...,...
5777,2018-12-31,red 11,7000,0
5778,1999-04-02,following,6000,240495
5779,2005-07-13,return to the land of wonders,5000,1338
5780,2015-09-29,a plague so pleasant,1400,0


,original_language,popularity,title,vote_average,vote_count
0,en,33.533,harry potter and the deathly hallows: part 1,7.7,10788
1,en,28.734,how to train your dragon,7.7,7610
2,en,28.515,iron man 2,6.8,12368
3,en,28.005,toy story,7.9,10174
4,en,27.920,inception,8.3,22186
...,...,...,...,...,...
26512,en,0.600,laboratory conditions,0.0,1
26513,en,0.600,_exhibit_84xxx_,0.0,1
26514,en,0.600,the last one,0.0,1
26515,en,0.600,trailer made,0.0,1


In [26]:
#Data cleaning to eliminate NaN values
#We will check for, and get rid of any NaN values in our dataset.

In [27]:
# check to see how many "NaN" values in df_imdb_edit
display(df_imdb_edit.isna().sum())
# drop the Nan rows 
df_imdb_edit = df_imdb_edit.dropna()
# check to see how many "NaN" values in df_tn_edit - NONE. No .dropna() needed.
display(df_tn_edit.isna().sum())
# check for 'Nan' values in df_tmdb)edit - NONE. No .dropna() needed.
display(df_tmdb_edit.isna().sum())

primary_title       0
genres           5352
dtype: int64

release_date         0
movie                0
production_budget    0
worldwide_gross      0
dtype: int64

original_language    0
popularity           0
title                0
vote_average         0
vote_count           0
dtype: int64

In [28]:

#Data cleaning to prepare for merge
#We will change "primary_title" in df_imdb_edit and "title" in df_tmdb_edit to "movie" to match df_tn_edit, and make the "movie" column name consistent across the 3 data sets

In [29]:
#We will change "primary_title" in df_imdb_edit and "title" in df_tmdb_edit to "movie" to match df_tn_edit, and make the "movie" column name consistent across the 3 data sets
df_imdb_edit = df_imdb_edit.rename(columns = {"primary_title": "movie"})
df_tmdb_edit = df_tmdb_edit.rename(columns = {"title": "movie"})

display(df_imdb_edit.head())
display(df_tmdb_edit.head())
display(df_tn_edit.head())

,movie,genres
0,sunghursh,"action,crime,drama"
1,one day before the rainy season,"biography,drama"
2,the other side of the wind,drama
3,sabse bada sukh,"comedy,drama"
4,the wandering soap opera,"comedy,drama,fantasy"


,original_language,popularity,movie,vote_average,vote_count
0,en,33.533,harry potter and the deathly hallows: part 1,7.7,10788
1,en,28.734,how to train your dragon,7.7,7610
2,en,28.515,iron man 2,6.8,12368
3,en,28.005,toy story,7.9,10174
4,en,27.920,inception,8.3,22186


,release_date,movie,production_budget,worldwide_gross
0,2009-12-18,avatar,425000000,2776345279
1,2011-05-20,pirates of the caribbean: on stranger tides,410600000,1045663875
2,2019-06-07,dark phoenix,350000000,149762350
3,2015-05-01,avengers: age of ultron,330600000,1403013963
4,2017-12-15,star wars ep. viii: the last jedi,317000000,1316721747


In [30]:
#Step 4: Merging Datasets

In [31]:

#Inner merge all datasets
#We will inner merge all 3 datasets on 'movie' column, because we want all rows to have data populated on movies that they all have in common.

In [32]:

#inner merge all 3 datasets on 'movie' column
df_merged = df_tmdb_edit.merge(df_tn_edit, on='movie').merge(df_imdb_edit, on='movie')
df_merged.head()

,original_language,popularity,movie,vote_average,vote_count,release_date,production_budget,worldwide_gross,genres
0,en,28.734,how to train your dragon,7.7,7610,2010-03-26,165000000,494870992,"action,adventure,animation"
1,en,28.515,iron man 2,6.8,12368,2010-05-07,170000000,621156389,"action,adventure,sci-fi"
2,en,27.920,inception,8.3,22186,2010-07-16,160000000,835524642,"action,adventure,sci-fi"
3,en,26.691,percy jackson & the olympians: the lightning t...,6.1,4229,2010-02-12,95000000,223050874,"adventure,family,fantasy"
4,en,26.526,avatar,7.4,18676,2009-12-18,425000000,2776345279,horror


In [33]:
#Sorting
#We will sort movies alphabetically, because the movies will become our index.

In [34]:
#sort dataframe by movies alphabetically
df_merged = df_merged.sort_values(by='movie', ascending=True)
df_merged.head()

,original_language,popularity,movie,vote_average,vote_count,release_date,production_budget,worldwide_gross,genres
2976,de,6.099,#horror,3.3,102,2015-11-20,1500000,0,"crime,drama,horror"
3180,en,17.892,10 cloverfield lane,6.9,4629,2016-03-11,5000000,108286422,"drama,horror,mystery"
3097,en,0.955,10 days in a madhouse,5.4,7,2015-11-11,12000000,14616,drama
3756,en,13.183,12 strong,5.6,1312,2018-01-19,35000000,71118378,"action,drama,history"
1895,en,16.493,12 years a slave,7.9,6631,2013-10-18,20000000,181025343,"biography,drama,history"


In [35]:
#Post-merge cleanup
#After merging, for any duplicate movies, we will only take the first instance where it shows up.

In [36]:
#run groupby and take only the first instance of each movie group
df_gp_movie = df_merged.groupby('movie')
df_gp_movie.first()
#reassign back to a dataframe object
df_merged_movie = df_gp_movie.first()
df_merged_movie

,original_language,popularity,vote_average,vote_count,release_date,production_budget,worldwide_gross,genres
movie,,,,,,,,
#horror,de,6.099,3.3,102,2015-11-20,1500000,0,"crime,drama,horror"
10 cloverfield lane,en,17.892,6.9,4629,2016-03-11,5000000,108286422,"drama,horror,mystery"
10 days in a madhouse,en,0.955,5.4,7,2015-11-11,12000000,14616,drama
12 strong,en,13.183,5.6,1312,2018-01-19,35000000,71118378,"action,drama,history"
12 years a slave,en,16.493,7.9,6631,2013-10-18,20000000,181025343,"biography,drama,history"
...,...,...,...,...,...,...,...,...
zipper,en,9.373,5.7,85,2015-08-28,4500000,0,"drama,thriller"
zookeeper,en,10.764,5.3,886,2011-07-08,80000000,170805525,"comedy,family,romance"
zoolander 2,en,12.997,4.7,1374,2016-02-12,50000000,55348693,comedy


In [37]:
df = df_merged_movie.reset_index()

In [38]:
df.head()

,movie,original_language,popularity,vote_average,vote_count,release_date,production_budget,worldwide_gross,genres
0,#horror,de,6.099,3.3,102,2015-11-20,1500000,0,"crime,drama,horror"
1,10 cloverfield lane,en,17.892,6.9,4629,2016-03-11,5000000,108286422,"drama,horror,mystery"
2,10 days in a madhouse,en,0.955,5.4,7,2015-11-11,12000000,14616,drama
3,12 strong,en,13.183,5.6,1312,2018-01-19,35000000,71118378,"action,drama,history"
4,12 years a slave,en,16.493,7.9,6631,2013-10-18,20000000,181025343,"biography,drama,history"


In [39]:
# #interested in movie release by month - create new column named "release_month"
# df['release_month'] = pd.DatetimeIndex(df['release_date']).month
# df.head()

In [40]:
df['profit'] = df['worldwide_gross'] -df['production_budget']
df['profit'] = df['profit'].astype(int)
df.head()

,movie,original_language,popularity,vote_average,vote_count,release_date,production_budget,worldwide_gross,genres,profit
0,#horror,de,6.099,3.3,102,2015-11-20,1500000,0,"crime,drama,horror",-1500000
1,10 cloverfield lane,en,17.892,6.9,4629,2016-03-11,5000000,108286422,"drama,horror,mystery",103286422
2,10 days in a madhouse,en,0.955,5.4,7,2015-11-11,12000000,14616,drama,-11985384
3,12 strong,en,13.183,5.6,1312,2018-01-19,35000000,71118378,"action,drama,history",36118378
4,12 years a slave,en,16.493,7.9,6631,2013-10-18,20000000,181025343,"biography,drama,history",161025343


In [41]:
# df.reset_index(drop=True)

In [42]:
df.head()

,movie,original_language,popularity,vote_average,vote_count,release_date,production_budget,worldwide_gross,genres,profit
0,#horror,de,6.099,3.3,102,2015-11-20,1500000,0,"crime,drama,horror",-1500000
1,10 cloverfield lane,en,17.892,6.9,4629,2016-03-11,5000000,108286422,"drama,horror,mystery",103286422
2,10 days in a madhouse,en,0.955,5.4,7,2015-11-11,12000000,14616,drama,-11985384
3,12 strong,en,13.183,5.6,1312,2018-01-19,35000000,71118378,"action,drama,history",36118378
4,12 years a slave,en,16.493,7.9,6631,2013-10-18,20000000,181025343,"biography,drama,history",161025343


In [43]:
data = df.drop(columns = ['movie','original_language','release_date'])

In [44]:
data.head()

,popularity,vote_average,vote_count,production_budget,worldwide_gross,genres,profit
0,6.099,3.3,102,1500000,0,"crime,drama,horror",-1500000
1,17.892,6.9,4629,5000000,108286422,"drama,horror,mystery",103286422
2,0.955,5.4,7,12000000,14616,drama,-11985384
3,13.183,5.6,1312,35000000,71118378,"action,drama,history",36118378
4,16.493,7.9,6631,20000000,181025343,"biography,drama,history",161025343


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1834 entries, 0 to 1833
Data columns (total 7 columns):
popularity           1834 non-null float64
vote_average         1834 non-null float64
vote_count           1834 non-null int64
production_budget    1834 non-null int64
worldwide_gross      1834 non-null int64
genres               1834 non-null object
profit               1834 non-null int64
dtypes: float64(2), int64(4), object(1)
memory usage: 100.4+ KB


In [46]:
# sns.pairplot(data)

In [47]:
data.head()

,popularity,vote_average,vote_count,production_budget,worldwide_gross,genres,profit
0,6.099,3.3,102,1500000,0,"crime,drama,horror",-1500000
1,17.892,6.9,4629,5000000,108286422,"drama,horror,mystery",103286422
2,0.955,5.4,7,12000000,14616,drama,-11985384
3,13.183,5.6,1312,35000000,71118378,"action,drama,history",36118378
4,16.493,7.9,6631,20000000,181025343,"biography,drama,history",161025343


In [48]:
# 'crime,drama,horror'.split(',')

In [49]:
variable_split = df['genres'].str.split(',')
variable_split

0               [crime, drama, horror]
1             [drama, horror, mystery]
2                              [drama]
3             [action, drama, history]
4          [biography, drama, history]
                     ...              
1829                 [drama, thriller]
1830         [comedy, family, romance]
1831                          [comedy]
1832          [comedy, drama, romance]
1833    [adventure, animation, comedy]
Name: genres, Length: 1834, dtype: object

In [50]:
type(variable_split)

pandas.core.series.Series

In [51]:
type(variable_split[0])

list

In [52]:
type(variable_split[0][0])

str

In [53]:
variable_split.str.get(0)

0           crime
1           drama
2           drama
3          action
4       biography
          ...    
1829        drama
1830       comedy
1831       comedy
1832       comedy
1833    adventure
Name: genres, Length: 1834, dtype: object

In [54]:
variable_split.str.get(1)

0           drama
1          horror
2             NaN
3           drama
4           drama
          ...    
1829     thriller
1830       family
1831          NaN
1832        drama
1833    animation
Name: genres, Length: 1834, dtype: object

In [55]:
variable_split.str.get(2)

0        horror
1       mystery
2           NaN
3       history
4       history
         ...   
1829        NaN
1830    romance
1831        NaN
1832    romance
1833     comedy
Name: genres, Length: 1834, dtype: object

In [56]:
data['genre'] = variable_split.str.get(0)
data['genre_1'] = variable_split.str.get(1)
data['genre_2'] = variable_split.str.get(2)

In [57]:
data.head()

,popularity,vote_average,vote_count,production_budget,worldwide_gross,genres,profit,genre,genre_1,genre_2
0,6.099,3.3,102,1500000,0,"crime,drama,horror",-1500000,crime,drama,horror
1,17.892,6.9,4629,5000000,108286422,"drama,horror,mystery",103286422,drama,horror,mystery
2,0.955,5.4,7,12000000,14616,drama,-11985384,drama,NaN,NaN
3,13.183,5.6,1312,35000000,71118378,"action,drama,history",36118378,action,drama,history
4,16.493,7.9,6631,20000000,181025343,"biography,drama,history",161025343,biography,drama,history


In [58]:
df_f = data.drop(columns =['genres'])
df_f.head()

,popularity,vote_average,vote_count,production_budget,worldwide_gross,profit,genre,genre_1,genre_2
0,6.099,3.3,102,1500000,0,-1500000,crime,drama,horror
1,17.892,6.9,4629,5000000,108286422,103286422,drama,horror,mystery
2,0.955,5.4,7,12000000,14616,-11985384,drama,NaN,NaN
3,13.183,5.6,1312,35000000,71118378,36118378,action,drama,history
4,16.493,7.9,6631,20000000,181025343,161025343,biography,drama,history


In [59]:
df_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1834 entries, 0 to 1833
Data columns (total 9 columns):
popularity           1834 non-null float64
vote_average         1834 non-null float64
vote_count           1834 non-null int64
production_budget    1834 non-null int64
worldwide_gross      1834 non-null int64
profit               1834 non-null int64
genre                1834 non-null object
genre_1              1509 non-null object
genre_2              1079 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 129.1+ KB


In [60]:
df_f.isna().sum()

popularity             0
vote_average           0
vote_count             0
production_budget      0
worldwide_gross        0
profit                 0
genre                  0
genre_1              325
genre_2              755
dtype: int64

In [61]:
data = df_f.dropna()

In [62]:
data.isna().sum()

popularity           0
vote_average         0
vote_count           0
production_budget    0
worldwide_gross      0
profit               0
genre                0
genre_1              0
genre_2              0
dtype: int64

In [68]:
data.shape

(1079, 9)

In [69]:
data.to_csv('imdb_ahsan.csv')

Save clean data.